In [1]:
import pandas as pd
import urllib, urllib.request
import requests
import time

In [4]:
#takes DOI id, can also use arxiv link or search with title
ids=["10.1145/2133806.2133826"]

In [57]:
missed=[]
ret=[]

for DOI in ids:
    url="https://api.semanticscholar.org/graph/v1/paper/DOI:"+DOI+"?fields=title,publicationDate,authors,s2FieldsOfStudy,citations,citations.authors"
    response = requests.get(url)
    try:
        print(len(response.json()))
    except:
        missed.append(url)
        print(url)
        #waits when timeout
        time.sleep(1)
        continue
    i=response.json()
    title=i["title"]
    date=i["publicationDate"]

    tags=[]
    for t in i["s2FieldsOfStudy"]:
        tags.append(t["category"])
    authors=[]
    for a in i["authors"]:
        authors.append(a["name"])
    cited_ids=[]
    cited_authors=[]


    for c in i["citations"]:
        cited_ids.append(c["paperId"])
    
    if len(cited_ids)==1000:
        #initialize queue with second set of articles
        urls=["https://api.semanticscholar.org/graph/v1/paper/DOI:"+DOI+"/citations?limit=1000&offset=1000"]
        while urls:
            
            url=urls.pop(0)
            
            #handle timeout
            try:
                response = requests.get(url)
            except: 
                #repeat call
                urls.append(url)
                time.sleep(1)
                continue
            
            i=response.json()
            for c in i["data"]:
                cited_ids.append(c["citingPaper"]["paperId"])
            if "next" in i:
                #add next offset citation query to queue
                next_url=url.split("&offset=")[0]+"&offset="+str(i["next"])
                urls.append(next_url)

    citation_count=len(cited_ids)
    ret.append([date,title,tags, authors,cited_ids,cited_authors, citation_count])

6


In [56]:
next_url

'https://api.semanticscholar.org/graph/v1/paper/DOI:10.1145/2133806.2133826/citations?limit=10002000'

In [46]:
urls

['https://api.semanticscholar.org/graph/v1/paper/DOI:10.1145/2133806.2133826/citations?offset=2000&limit=1000']

In [31]:
url=

In [32]:
response = requests.get(url)

In [33]:
i=response.json()

In [58]:
out=pd.DataFrame(ret)

In [59]:
out.columns=["date", "title", "tags", "authors", "cited_ids", "cited_authors", "citation_count"]

In [60]:
out.to_csv("semantic_meta.csv")

In [61]:
len(out["cited_ids"][0])

5476